In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configurations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
# Hyper-parameters
num_epochs = 3
batch_size = 100
num_classes = 10
learning_rate=0.001

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="../../data", train=False, transform=transforms.ToTensor(), download=False)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# define Net
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # out = [bs, w=14, h=14, features=16] 
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # out = [bs, w=7, h=7, features=32] 
        
        self.fc = nn.Linear(in_features=7*7*32, out_features=num_classes)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1) # This is a good operation, u should learn this
        out = self.fc(out)
        return out

In [6]:
# Init Net, Loss, Optimizer
model = ConvNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
for param in model.parameters():
    #print(param.shape)
    pass
    """
    torch.Size([16, 1, 5, 5])  # 16 filters,  size of filer is [depth=1, h=5, w=5]
    torch.Size([16])
    torch.Size([16])
    torch.Size([16])
    torch.Size([32, 16, 5, 5]) # 32 filters,  size of filer is [depth=16, h=5, w=5]
    torch.Size([32])
    torch.Size([32])
    torch.Size([32])
    torch.Size([10, 1568]) # 10 rows, each row has 7*7*32 length items to fully connect the outputs
    torch.Size([10])    # 10 bias
    """

In [13]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # get INPUT&GT
        images = images.to(device)
        labels = labels.to(device)
        
        # OUTPUT: forward through NET & LOSS
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Zero & Backward & Step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Logging
        if (i+1) %200 == 0:
            print('epoch[{}/{}],  step[{}/{}] loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

epoch[1/3],  step[200/600] loss: 0.0408
epoch[1/3],  step[400/600] loss: 0.0525
epoch[1/3],  step[600/600] loss: 0.0311
epoch[2/3],  step[200/600] loss: 0.0370
epoch[2/3],  step[400/600] loss: 0.0268
epoch[2/3],  step[600/600] loss: 0.0592
epoch[3/3],  step[200/600] loss: 0.0585
epoch[3/3],  step[400/600] loss: 0.0591
epoch[3/3],  step[600/600] loss: 0.0194


In [14]:
# Test the model
model.eval()
total_step = len(test_loader)
with torch.no_grad():
    correct=0
    total=0
    for i,(images, labels) in enumerate(test_loader):
        # get INPUT & GT
        images = images.to(device)
        labels = labels.to(device)
        # forward through Net
        outputs = model(images)
        # extract PREDICTION 
        _, predicted = torch.max(outputs.data, 1)
        # compare GT & PREDICTION
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # Logging
        if (i+1) %10 == 0:
            print('Step[{}/{}] Accuracy: {:.4f}'.format(i+1,total_step,100.0*correct/total))

Step[10/100] Accuracy: 98.6000
Step[20/100] Accuracy: 98.3500
Step[30/100] Accuracy: 98.3000
Step[40/100] Accuracy: 98.4250
Step[50/100] Accuracy: 98.5800
Step[60/100] Accuracy: 98.7333
Step[70/100] Accuracy: 98.8143
Step[80/100] Accuracy: 98.9250
Step[90/100] Accuracy: 99.0222
Step[100/100] Accuracy: 99.0400


In [22]:
torch.save(model.state_dict(), 'model.ckpt')